### steps
1. Create an sample SQL database
2. create config.ini for Mysql database connection
3. create table
4. performing SQL window on database to solve department highest salary


In [2]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config # Read database configuration file and return a dictionary database object